In [1]:
import pandas as pd

import json
import glob
import os
import re

from shared import *

In [2]:
base = "../exp/"
paths = glob.glob(base + "data/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    **algo,
    'graph': path_to_graph(run['args'][1]),
    'metric': run['live_weight_file'],
    'queries': run['queries'],
} for run in data for algo in run['algo_runs']])
queries['nontrivial'] = queries['num_iterations'] > 1

In [4]:
queries_sub = queries.query("~(graph == 'OSM Germany' & metric == 'fake_traffic')")
queries_sub.query("~failed").groupby(['queries', 'graph', 'metric', 'algo']) \
    .mean()[['length_increase_percent', 'num_iterations', 'num_forbidden_paths', 'running_time_ms']] \
    .join(queries_sub.groupby(['queries', 'graph', 'metric', 'algo']).mean()[['failed']]) \
    .unstack().swaplevel(axis='columns').sort_index(axis='columns')

algo                                        iterative_detour_blocking  \
                                                               failed   
queries         graph         metric                                    
queries/1h      DIMACS Europe fake_traffic                       0.05   
                OSM Europe    fake_traffic                       0.00   
                OSM Germany   heavy_traffic                      0.00   
                              lite_traffic                       0.00   
queries/4h      OSM Europe    fake_traffic                       0.00   
                OSM Germany   heavy_traffic                      0.01   
                              lite_traffic                       0.00   
queries/uniform DIMACS Europe fake_traffic                       0.01   
                OSM Europe    fake_traffic                       0.00   
                OSM Germany   heavy_traffic                      0.02   
                              lite_traffic                       0.01   

algo                                                                 \
                                            length_increase_percent   
queries         graph         metric                                  
queries/1h      DIMACS Europe fake_traffic                 0.871700   
                OSM Europe    fake_traffic                 0.221528   
                OSM Germany   heavy_traffic                2.251564   
                              lite_traffic                 0.825125   
queries/4h      OSM Europe    fake_traffic                 0.105107   
                OSM Germany   heavy_traffic                0.597471   
                              lite_traffic                 0.207796   
queries/uniform DIMACS Europe fake_traffic                 0.695059   
                OSM Europe    fake_traffic                 0.154250   
                OSM Germany   heavy_traffic                2.416044   
                              lite_traffic                 1.472414   

algo                                                             \
                                            num_forbidden_paths   
queries         graph         metric                              
queries/1h      DIMACS Europe fake_traffic             2.905263   
                OSM Europe    fake_traffic             1.150000   
                OSM Germany   heavy_traffic           85.460000   
                              lite_traffic            16.130000   
queries/4h      OSM Europe    fake_traffic             1.770000   
                OSM Germany   heavy_traffic           86.353535   
                              lite_traffic            13.840000   
queries/uniform DIMACS Europe fake_traffic             2.303030   
                OSM Europe    fake_traffic             1.090000   
                OSM Germany   heavy_traffic          125.214286   
                              lite_traffic            40.424242   

algo                                                                        \
                                            num_iterations running_time_ms   
queries         graph         metric                                         
queries/1h      DIMACS Europe fake_traffic        2.284211        1.585381   
                OSM Europe    fake_traffic        1.280000        0.939329   
                OSM Germany   heavy_traffic      12.460000      281.848370   
                              lite_traffic        3.200000       20.243618   
queries/4h      OSM Europe    fake_traffic        1.630000        7.893662   
                OSM Germany   heavy_traffic      11.212121      632.944341   
                              lite_traffic        3.170000       45.157303   
queries/uniform DIMACS Europe fake_traffic        2.212121        0.774327   
                OSM Europe    fake_traffic        1.440000        3.230204   
                OSM Germany   heavy_traffic      16.357143      641.336560   
                             